In [1]:
import pandas as pd

df = pd.read_excel (r'/content/data_berita.xlsx')
print(df)

    index                                                url  ...  topic  sentiment
0       0  https://news.detik.com/berita/d-5557626/gugata...  ...    NaN        NaN
1       1  https://news.detik.com/berita/d-5555995/batalk...  ...    NaN        NaN
2       2  https://news.detik.com/berita/d-5558352/10-rib...  ...    NaN        NaN
3       3  https://news.detik.com/detiktv/d-5559464/ini-m...  ...    NaN        NaN
4       4  https://news.detik.com/berita/d-5555665/satpol...  ...    NaN        NaN
5       5  https://news.detik.com/berita/d-5555608/andi-a...  ...    NaN        NaN
6       6  https://news.detik.com/berita/d-5555898/cegah-...  ...    NaN        NaN
7       7  https://news.detik.com/berita/d-5554245/kala-a...  ...    NaN        NaN
8       8  https://news.detik.com/berita/d-5556696/jangan...  ...    NaN        NaN
9       9  https://news.detik.com/berita/d-5557585/manaje...  ...    NaN        NaN
10     10  http://megapolitan.kompas.com/read/2021/05/04/...  ...    NaN    

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import string
import urllib.request
import re

stopword = []
stopword_data = urllib.request.urlopen('http://static.hikaruyuuki.com/wp-content/uploads/stopword_list_tala.txt').read().decode('utf-8').split('\n')
for line in stopword_data:
    stopword.append(line)

# cleaning symbol
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\r', '', text)
    text = ' '.join([word for word in text.split() if word.lower() not in stopword])
    return text    

news = ['detik','kompas']
i = 0
for url in df['url']:
    newssource = 0
    # get news source type
    for new in news:
        if url.find(new) != -1:
          newssource = new
    if newssource == 0:
        print('News source is not detected!')

    # get page text
    page = requests.get(url)
    
    # parse with BFS
    soup = BeautifulSoup(page.text, 'html.parser')    

    # get article title
    atitle = soup.find('title').text 

    # get article author
    if newssource == 'detik':
        aauthor = str(soup.find('div', {'class': 'detail__author'}))
    if newssource == 'kompas':
        aauthor = str(soup.find('span', {'class': 'pb_10 author'}))
    if aauthor != 'None':
        aauthor = aauthor[aauthor.index('">')+2:aauthor.index('</')]

    # get article date
    if newssource == 'detik':
        adate = str(soup.find('div', {'class': 'detail__date'}))
    if newssource == 'kompas':
        adate = str(soup.find('div', {'class': 'read__time'}))
    if newssource != 'None':
        adate = adate[adate.index('">')+2:adate.index('</')]  


    # get main article page
    if newssource == 'detik':
        articlebody = soup.find(class_='detail__body-text itp_bodycontent')
    if newssource == 'kompas':
        articlebody = soup.find(class_='read__content')
    # get text
    articletext = articlebody.find_all('p')

    # store the text for each article
    paragraphtext = [] 
    for paragraph in articletext:
        # get the text only
        text = paragraph.get_text()
        text = clean_text(text)
        paragraphtext.append(text)

    # combine all paragraphs into an article
    df['news'][i] = ' '.join(paragraphtext)
    df['title'][i] = atitle
    df['author'][i] = aauthor
    df['date'][i] = adate
    i +=1


In [3]:
df

,index,url,news,title,author,date,jakarta,topic,sentiment
0,0,https://news.detik.com/berita/d-5557626/gugata...,majelis hakim pengadilan negeri jakarta pusat ...,"Gugatan soal AD/ART Gugur, Kubu Moeldoko Dinil...",Tim detikcom - detikNews,"Selasa, 04 Mei 2021 17:47 WIB",NaN,NaN,NaN
1,1,https://news.detik.com/berita/d-5555995/batalk...,habib rizieq mengaku membatalkan acaranya berk...,"Batalkan Keliling Indonesia, Habib Rizieq Klai...",Dwi Andayani - detikNews,"Senin, 03 Mei 2021 16:23 WIB",NaN,NaN,NaN
2,2,https://news.detik.com/berita/d-5558352/10-rib...,stasiun pasar senen jakarta pusat dipadati pen...,10 Ribu Penumpang Berangkat dari Stasiun Senen...,Wilda Hayatun Nufus - detikNews,"Rabu, 05 Mei 2021 10:16 WIB",NaN,NaN,NaN
3,3,https://news.detik.com/detiktv/d-5559464/ini-m...,polisi menyita kendaraan berpelat biru nomor p...,"Ini Mobil Bernopol Nyeleneh, Pengemudi Ngaku J...",detikTV - detikNews,"Rabu, 05 Mei 2021 20:42 WIB",NaN,NaN,NaN
4,4,https://news.detik.com/berita/d-5555665/satpol...,kerumunan konser musik cibis park pasar minggu...,Satpol PP: Penyelenggara Konser Musik Izin ke ...,Adhyasta Dirgantara - detikNews,"Senin, 03 Mei 2021 14:00 WIB",NaN,NaN,NaN
5,5,https://news.detik.com/berita/d-5555608/andi-a...,politikus andi arief fadli zon tokoh menjamin ...,"Andi Arief, Fadli Zon dan 18 Tokoh Jadi Penjam...",Eva Safitri - detikNews,"Senin, 03 Mei 2021 13:18 WIB",NaN,NaN,NaN
6,6,https://news.detik.com/berita/d-5555898/cegah-...,perumda pasar jaya menetapkan aturan jam tutup...,"Cegah Kerumunan, Jam Tutup Pasar Tanah Abang L...",Wilda Hayatun Nufus - detikNews,"Senin, 03 Mei 2021 15:30 WIB",NaN,NaN,NaN
7,7,https://news.detik.com/berita/d-5554245/kala-a...,cerita babi ngepet menghebohkan wraga bedahan ...,Kala Adam Terseret dalam Jebakan Kebohongan Ba...,Tim detikcom - detikNews,"Minggu, 02 Mei 2021 03:01 WIB",NaN,NaN,NaN
8,8,https://news.detik.com/berita/d-5556696/jangan...,pemerintah melaporkan mutasi virus corona indi...,Jangan Abai Prokes! Mutasi Corona dari India D...,Lisye Sri Rahayu - detikNews,"Selasa, 04 Mei 2021 08:32 WIB",NaN,NaN,NaN
9,9,https://news.detik.com/berita/d-5557585/manaje...,polisi memeriksa saksi terkait kerumunan konse...,Manajemen Cibis Park hingga EO Diperiksa Polis...,Luqman Nurhadi Arunanta - detikNews,"Selasa, 04 Mei 2021 17:28 WIB",NaN,NaN,NaN
